In [ ]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 47.8 MB/s eta 0:00:00


In [ ]:
pip install scikit-learn

In [ ]:
pip install gensim scikit-learn

In [ ]:
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

In [ ]:
# 필요한 라이브러리를 임포트
import os
from xml.etree import ElementTree
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
import os
from xml.etree import ElementTree
import json

# 폴더 내의 모든 파일 경로를 가져오는 함수
def get_file_paths(directory):
    file_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))]
    return file_paths

def extract_korean_texts(directory):
    file_paths = get_file_paths(directory)  # 디렉토리 내의 모든 파일 경로를 가져옴
    texts = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = json.load(file)
            if 'facts' in content:
                # 'facts' 부분만 문자열로 변환하여 추가
                facts_text = json.dumps(content['facts'], ensure_ascii=False)
                texts.append(facts_text)
    return texts


# 폴더 경로 지정
civil_case_directory = '/content/drive/MyDrive/SKT_FLY_AI/Project/civil'  # 민사 사건 파일이 있는 폴더 경로
criminal_case_directory = '/content/drive/MyDrive/SKT_FLY_AI/Project/criminal'  # 형사 사건 파일이 있는 폴더 경로

# 데이터 로드
civil_texts = extract_korean_texts(civil_case_directory)
criminal_texts = extract_korean_texts(criminal_case_directory)

# 민사 및 형사 텍스트 데이터를 하나의 데이터셋으로 결합
texts = civil_texts + criminal_texts  # 전체 텍스트 데이터
labels = ['민사'] * len(civil_texts) + ['형사'] * len(criminal_texts)  # 민사: 0, 형사: 1

from konlpy.tag import Okt

# Okt 객체를 생성합니다.
okt = Okt()

# 사용자 정의 불용어 리스트를 정의합니다.
stop_words = ['는', '의', '에', '을', '를', '과', '와', '이', '가', '도', '으로', '로', '에서', '하다', '형사', '민사', '형법', '민법']

# 민사 및 형사 텍스트가 저장된 변수를 정의합니다.
# civil_texts = ["민사 예시 텍스트 데이터", "민사 텍스트의 또 다른 예시"]
# criminal_texts = ["형사 예시 텍스트 데이터", "형사 텍스트의 또 다른 예시"]

# 텍스트를 토큰화합니다.
civil_tokens = [okt.morphs(text) for text in civil_texts]
criminal_tokens = [okt.morphs(text) for text in criminal_texts]

# 불용어를 제거합니다.
civil_processed = [[word for word in tokens if word not in stop_words] for tokens in civil_tokens]
criminal_processed = [[word for word in tokens if word not in stop_words] for tokens in criminal_tokens]

# 각 리스트의 단어들을 집합으로 변환합니다.
civil_word_set = set(word for sublist in civil_processed for word in sublist)
criminal_word_set = set(word for sublist in criminal_processed for word in sublist)

# 두 집합의 교집합(공통 단어)을 찾습니다.
common_words = civil_word_set.intersection(criminal_word_set)

# 공통 단어를 제거합니다.
civil_final = [[word for word in sublist if word not in common_words] for sublist in civil_processed]
criminal_final = [[word for word in sublist if word not in common_words] for sublist in criminal_processed]

# 결과를 변수에 저장합니다.
civil_texts = [' '.join(tokens) for tokens in civil_final]
criminal_texts = [' '.join(tokens) for tokens in criminal_final]



# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.25, random_state=42)

# 파이프라인 구축: TF-IDF 벡터라이저와 로지스틱 회귀 분류기
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', LogisticRegression(solver='liblinear')),
])

# 모델 훈련
pipeline.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = pipeline.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# 결과 출력
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy: 0.9555555555555556
Classification Report:
               precision    recall  f1-score   support

          민사       0.94      0.94      0.94        18
          형사       0.96      0.96      0.96        27

    accuracy                           0.96        45
   macro avg       0.95      0.95      0.95        45
weighted avg       0.96      0.96      0.96        45



In [ ]:
# Word2Vec 모델을 훈련시킵니다. 여기서는 이미 토큰화된 데이터를 사용합니다.
model_w2v = Word2Vec(sentences=civil_tokens + criminal_tokens, vector_size=100, window=5, min_count=1, workers=4)

# 단어 벡터의 평균을 구하는 함수를 정의합니다.
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=100):
    if len(tokens_list) < 1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

# 각 텍스트에 대한 평균 Word2Vec 벡터를 계산합니다.
X = np.array([get_average_word2vec(tokens, model_w2v.wv, generate_missing=True) for tokens in civil_tokens + criminal_tokens])

# 레이블을 생성합니다.
labels = [0] * len(civil_texts) + [1] * len(criminal_texts)  # 0은 민사, 1은 형사

# 데이터를 훈련 세트와 테스트 세트로 분할합니다.
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)

# 로지스틱 회귀 분석 모델을 생성하고 훈련합니다.
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

# 테스트 데이터에 대한 예측을 수행합니다.
y_pred = model_lr.predict(X_test)

# 정확도와 분류 리포트를 출력합니다.
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7407407407407407
              precision    recall  f1-score   support

           0       0.90      0.41      0.56        22
           1       0.70      0.97      0.82        32

    accuracy                           0.74        54
   macro avg       0.80      0.69      0.69        54
weighted avg       0.78      0.74      0.71        54



In [ ]:
new_example_file_path = '/content/drive/MyDrive/SKT_FLY_AI/Project/86가합493.json'
pred_texts = []
with open(new_example_file_path, 'r', encoding='utf-8') as file:
            new_example_text = json.load(file)
            if 'facts' in new_example_text:
                # 'facts' 부분만 문자열로 변환하여 추가
                facts_text = json.dumps(new_example_text['facts'], ensure_ascii=False)
                texts.append(facts_text)

pred_texts.append(json.dumps(new_example_text, ensure_ascii=False))

new_example_prediction = pipeline.predict(pred_texts)

# 예측 결과 출력
print("예측된 분류:", new_example_prediction[0])


예측된 분류: 민사


In [ ]:
new_example_file_path = '/content/drive/MyDrive/SKT_FLY_AI/Project/2010고단3085.json'
pred_texts = []
with open(new_example_file_path, 'r', encoding='utf-8') as file:
            new_example_text = json.load(file)
            if 'facts' in new_example_text:
                # 'facts' 부분만 문자열로 변환하여 추가
                facts_text = json.dumps(new_example_text['facts'], ensure_ascii=False)
                texts.append(facts_text)

pred_texts.append(json.dumps(new_example_text, ensure_ascii=False))

new_example_prediction = pipeline.predict(pred_texts)

# 예측 결과 출력
print("예측된 분류:", new_example_prediction[0])


예측된 분류: 형사


In [ ]:
new_text = ["피고인은 조현병으로 인하여 사물을 변별할 능력이나 의사를 결정할 능력이 미약한 상태에서 사람을 죽였다."]

new_example_prediction2 = pipeline.predict(new_text)

# 예측 결과 출력
print("예측된 분류:", new_example_prediction2[0])

예측된 분류: 형사


In [ ]:
new_text = ["내가 화가 나서 옆사람을 칼로 찔렀어."]

new_example_prediction2 = pipeline.predict(new_text)

# 예측 결과 출력
print("예측된 분류:", new_example_prediction2[0])

예측된 분류: 형사


In [ ]:
new_text = ["나 연예인인데 소속사가 계약위반을 했어."]

new_example_prediction2 = pipeline.predict(new_text)

# 예측 결과 출력
print("예측된 분류:", new_example_prediction2[0])

예측된 분류: 형사
